# 超参数与模型验证

## 首先对iris数据集进行K近邻处理

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

In [4]:
model.fit(X, y)
y_model = model.predict(X)

In [6]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_model)

1.0

## 由于将所有数据集都当作了训练集，导致经验误差为0，但是是不可能的，接下来进行数据集的切割，分为训练集和测试集

In [13]:
from sklearn.model_selection import train_test_split
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, test_size=0.5)
model.fit(X1, y1)
y2_model = model.predict(X2)
accuracy_score(y2, y2_model)

0.9066666666666666

## 交叉验证
上述方法虽然不错，但是有一个问题：降低了训练集体量，导致部分数据对模型没有做出任何贡献，下面做一个改进，让每个子集既是训练集也是验证集

In [15]:
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

(0.96, 0.9066666666666666)

## 自动化进行交叉验证训练
cv=5是指将样本五等分，每次以其中4份作为训练集，剩下一份作为测试集

In [17]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

每次只留一个样本作为测试，cv=LeaveOneOut(...)

In [21]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [22]:
scores.mean()

0.96

## 选择最优模型(偏差与方差的平衡)
机器学习最重要的部分，如何选择模型和超参数
一个模型判定准则：

判定系数$R^2$  （Coefficient_of_determination）

当取值为1表示模型与数据完全吻合

当取值为0表示模型不比简单取均值好

当取值为负表示模型性能很差

## 模型、参数自动化选择工具：Grid Search
form sklearn.grid_search import GridSearchCV